## TaskOperation & PackTask

@Author: 吴炜坤

@email：weikun.wu@xtalpi.com/weikunwu@163.com

上一节，我们详细介绍了Rotamer以及Rosetta Packer的基本工作原理。Packer有三种工作方式: “Repacking”、“Rotamer Trial”和“Design”。其中Rotamer就有可能存在3种基本状态:
- Repacking: 该位点处Rotamer类型只能是同一种氨基酸;
- Design: 该位点上Rotamer类型可以是多种氨酸;
- Fixed: 该位点Rotamer不允许发生变化。

在本章节中，读者将学习到如何使用TaskOperations来控制Rotamer在Packer中的“行为规范”。

### 一、PackerTask与TaskFactory

如果把Packer想象成一位“蛋白构建大师”，他正在将一些建筑材料(Rotamer)安装在地基上(Backbone的$C_{\alpha}$原子)。**但是在施工之前，建筑大师必须知道他需要在每个地基点上能安装什么材料**，并且经过他的深思熟虑(模拟退火)来构建出最完美的艺术品。此时，他需要我们的帮助，给他一张施工的蓝图。而这张蓝图就是PackerTask。

<center><img src="./img/PackerTask.jpg" width = "700" height = "200" align=center /></center>

In [33]:
# 生成含有5个精氨酸的Pose
from pyrosetta import *
pose = pose_from_sequence('RRRRR')

**一般的PackerTask需要从TaskFactory中进行自动生成。**

In [34]:
# 从TaskFactory生成packer_task
from pyrosetta.rosetta.core.pack.task import TaskFactory
tf = TaskFactory()
packer_task = tf.create_packer_task(pose)
print(packer_task)

#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	TRUE	ALA:NtermProteinFull,CYS:NtermProteinFull,ASP:NtermProteinFull,GLU:NtermProteinFull,PHE:NtermProteinFull,GLY:NtermProteinFull,HIS:NtermProteinFull,HIS_D:NtermProteinFull,ILE:NtermProteinFull,LYS:NtermProteinFull,LEU:NtermProteinFull,MET:NtermProteinFull,ASN:NtermProteinFull,PRO:NtermProteinFull,GLN:NtermProteinFull,ARG:NtermProteinFull,SER:NtermProteinFull,THR:NtermProteinFull,VAL:NtermProteinFull,TRP:NtermProteinFull,TYR:NtermProteinFull
2	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
3	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
4	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
5	TRUE	TRUE	ALA:CtermProteinFull,CYS:CtermProteinFull,ASP:CtermProteinFull,GLU:CtermProteinFull,PHE:CtermProteinFull,GLY:CtermProteinFull,HIS:CtermProteinFull,HIS_D:Cterm

从上述结果可见，默认生成的Packer_Task包含三种数据:
- 残基位点编号信息
- 每个位点上，Rotamer的3种基本状态
- 每个位点上，被允许氨基酸Rotamer类型

**并且默认的PackerTask是运行每个位点都包含20种氨基酸的Rotamer**

### 二、什么是TaskOperations?

TaskOperations直译的意思即为"任务操作元件", **TaskOperations被TaskFactory加载并创建PackerTask。**

TaskOperations可以直观的理解为对**指定位点氨基酸的Rotamer自由度进行消减的过程**，类似做雕刻的过程，一开始默认的PackerTask允许所有20种氨基酸的Rotamer的出现，在加载不同的TaskOperations后，PackerTask的Rotamer自由度开始不断的缩减，直到满足用户设定的需求。其中一点是非常重要的。**一旦将Rotamer的自由度缩减后，将无法在PackerTask中重新被激活。** 如，第一个TaskOperations设定5号位点为“Fixed“的状态，第二个TaskOperations再重新设定5号位点为“Repacking“的状态。TaskFactory生成的PackerTask中5号位点**依然为“Fixed(No_repack)“的状态**。从底层逻辑来理解，TaskFactory将所有的TaskOperations中取Rotamer自由度越小的那个状态来生成PackerTask。

<center><img src="./img/taskop_turnoff.png" width = "500" height = "200" align=center /></center>

小结: 
* TaskOperations的三层自由度级别: Design/Repack/No_repack;
* TaskOperations形象地类似于"冰雕设计", 它的作用是告诉TaskFactory在Pose中每个位点的Rotamer自由度大小;
* 一旦一个氨基酸或区域被设计定为低自由度状态时，那么这个位点将不能被重新设定为高自由度状态。

### 三、TaskOperations的分类

TaskOperation从构建逻辑上来分类共计有两种类型:

* Residue Level TaskOperations: 根据Selector设定选择范围内位点的Rotamer自由度(手动挡);
* Specialized Operations: 根据预设好逻辑，对位点进行全局Rotamer操作(自动挡);

#### 3.1 Residue Level TaskOperations的使用示例

Residue Level TaskOperations(RLT)一般需要配合Selector来指定操作的范围。用户可以直观地将RLT理解为一个**自定义版本的Specialized Operations**。

**特别注意的是:RLT是无法直接被TaskFactory所读取，其必须通过OperateOnResidueSubset函数来生成一个标准的TaskOperations。**

此处举一个简单RestrictToRepackingRLT(顾明思议此RTL是将Rotamer约束到现有氨基酸类型上。)应用的例子：

In [44]:
from pyrosetta.rosetta.core.pack.task.operation import RestrictToRepackingRLT, OperateOnResidueSubset
from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector
# 选择氨基酸范围
select_pos3 = ResidueIndexSelector('1,3,5')

# 使用OperateOnResidueSubset生成TaskOperations
packing_taskop = OperateOnResidueSubset(RestrictToRepackingRLT(), select_pos3, False)

# 将TaskOperations加载至TaskFactory中
pack_tf = TaskFactory()
pack_tf.push_back(packing_taskop)

# 生成PackerTask
packer_task = pack_tf.create_task_and_apply_taskoperations(pose)
print(packer_task)

#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	FALSE	ARG:NtermProteinFull
2	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
3	TRUE	FALSE	ARG
4	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
5	TRUE	FALSE	ARG:CtermProteinFull



此时，通过PackerTask我们可以观测到，我们选择的1,3,5号氨基酸都只被允许为进行pack，氨基酸的类型也只剩下ARG。

除了RestrictToRepackingRLT以外，还有许多其他7种常用的RTL类型，此处做一个简单的列表解释用途:
- RestrictToRepackingRLT: 将Rotamer自由度设置为repacking
- PreventRepackingRLT: 将Rotamer自由度设置为no_repack
- RestrictAbsentCanonicalAASExceptNativeRLT: 将Rotamer自由度限定在给定的氨基酸类型列表，并允许保留当前位点氨基酸类型的Rotamer保留
- RestrictAbsentCanonicalAASRLT: 将Rotamer自由度限定在给定的氨基酸类型列表。
- DisallowIfNonnativeRLT: 将Rotamer自由度限定在**非给定的氨基酸类型列表**，并允许保留当前位点氨基酸类型的Rotamer保留
- IncludeCurrentRLT: 设定Packer在执行期间，考虑Pose输入时的Rotamer状态(并非全部忘记)。
- ExtraRotamersGenericRLT: 设定在Rotamer采样中，是否考虑增加额外的数量，否则仅会考虑理想型的Rotamer。 

上述的这些RTL和Selector就可以组合成任意的TaskOperations，更多RLT的API用法请参考TaskOperations API详解相关的章节。

#### 思考:
如果将OperateOnResidueSubset中的False转变为True，PackerTask会发生什么变化？

#### 3.2 Specialized Operations的使用示例

Specialized Operations其实就是开发者预先设定好一些应用场景的TaskOperations，这类TaskOperations可以直接被TaskFactory所读取，一般的这种TaskOperations的应用范围都是全局性。

In [54]:
from pyrosetta.rosetta.core.pack.task.operation import RestrictToRepacking

# 将TaskOperations加载至TaskFactory中
pack_tf = TaskFactory()
pack_tf.push_back(RestrictToRepacking())

# 生成PackerTask
packer_task = pack_tf.create_task_and_apply_taskoperations(pose)
print(packer_task)

#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	FALSE	SER:NtermProteinFull
2	TRUE	FALSE	GLY
3	TRUE	FALSE	GLY
4	TRUE	FALSE	GLY
5	TRUE	FALSE	SER:CtermProteinFull



可见所有位点都被限制为repacking的状态，更多Specialized Operations的API用法请参考TaskOperations API详解相关的章节。

### 四、与Pack Rotamer相关Mover

上述内容已经清楚地阐释了PackerTask的作用，以及如何去利用TaskOperations和TaskFactory去生成特定用途的PackerTask。当有了这张清晰的“蓝图文件后”，就实际地在Pose结构上进行“施工”，而真正执行任务的Mover大多可以在pyrosetta.rosetta.protocols.minimization_packing的API下找到。此处我们以最简单的PackRotamersMover做实例示范。

In [59]:
from pyrosetta.rosetta.protocols.minimization_packing import PackRotamersMover
from pyrosetta import create_score_function
pack_mover = PackRotamersMover()
ref2015 = create_score_function('ref2015')
pose = pose_from_sequence('RRRRR')
pose.dump_pdb('./data/orignal.pdb')

# 不需要导入PackTask，只需要输入TaskFactory即可。
pack_mover.task_factory(pack_tf)
pack_mover.score_function(ref2015)

# 执行repacking
pack_mover.apply(pose)
pose.dump_pdb('./data/repacked.pdb')

core.pack.pack_rotamers: {0} built 74 rotamers at 5 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).


对比原始输入结构和repack之后的结构变化:
<center><img src="./img/repacking.png" width = "400" height = "200" align=center /></center>

上述的介绍的PackRotamersMover所做的就仅仅是处理侧链构象相关的模拟退火过程。但是在实际应用中Rosetta常常交替式运行Packer和Minimizer(MinMover)，这类组合型的Mover常有的有FastRelaxMover以及FastDesignMover。

一张图看懂FastRelax在做什么: